![Image of Yaktocat](http://yuml.me/c961b9c3)

#States
* INIT
  * reads underlined text
  * extracts context
    * set next state = PREAMBLE
    
* PREAMBLE
  * reads PREAMBLE (NOTICE IS HEREBY...)
  * extracts preamble
    * set next state = PROCESS_ADVERTS

* PROCESS_ADVERTS
  * reads first key and first chunk of value
    * set next state = PROCESS_ADVERT
    
* PROCESS_ADVERT
  * reads next chunk and evaluate.
    ```
    if ':' in chunk then 
       store current key/value
       set current key, and initial value
    else
       append chunk to current-value
    ```



In [ ]:
from bs4 import BeautifulSoup as Soup
from IPython.display import HTML
import pandas as pd
import numpy as np
import re
import pprint
from IPython.display import display

In [ ]:
class State:
    def run(self):
        # implement this for printing trace statements
        assert 0, "implement me"
        
    def next(self, input):
        assert 0, "implement me"
        
    def flush(self, record):
        raise Exception('wtf!')



In [ ]:
class Adverts(State):
    
    def __init__(self):
        State.__init__(self)
        
        # Current key and Value
        # value can span multiple lines/paragraphs
        # so we have to build it up a chunk at a time.
        # The complete text of value is assumed when
        # a new key/value pair is detected
        self.key = None
        self.value = None
        
        # Current Advertisement
        self.advert = {}
        self.processingAdvert = False
        
    def run(self):
        print('Process ADVERTS')
        
    def flush(self, record):
        self.advert[self.key] = self.value.strip()
        record['adverts'].append(self.advert)
        
    def next(self, para, record):
        text = para.get_text(strip=True)
        if ':' in text:
            k, v = text.split(':', 1)
            k = k.strip()
            if k.lower().startswith('agency'):
                # the first time the code gets here
                # processingAdvert is False, set it to True
                # so that from now on getting here means
                # we've completed an Advertisement
                
                if self.processingAdvert:
                    # End of record reached
                    self.advert[self.key] = self.value.strip()
                    record['adverts'].append(self.advert)
                    self.advert = {}
                
                # yep, processing advertisements
                self.processingAdvert = True
                    
            else:
                # A new key/value pair is detected,
                # store current key/value pair
                self.advert[self.key] = self.value.strip()
                
            # Initialize the current key/value pair
            self.key, self.value = k, v
            
        else:
            # A value can span multiple lines
            # keep adding chunks to the current value
            # until a new key/value pair is detected...
            if self.value:
                self.value += text   # Append to a previous value
            else:
                self.value = text    # Otherwise set value
            
        return self
        

class Preamble(State):
    def run(self):
        print('PREAMBLE')
        
    def next(self, para, record):
        text = para.get_text(strip=True).strip()
        if 'NOTICE IS HEREBY' in text:
            record['preamble'] = text
            return Adverts()
        return self
            

class Init(State):
    def run(self):
        print('INIT')
        
    def next(self, para, record):
        if para.find_all('u'):
            record['context'] = para.get_text(strip=True).strip()
            return Preamble()
        return self

        

In [ ]:
def parse_notice(soup):
    state = Init()
    record = { 'adverts': []}

    for para in soup.find_all('p'):
    #     state.run()
    #     print (para)
        state = state.next(para, record)
    #     print ('\n\n')

    state.flush(record)
    return record



In [ ]:
fn = 'orig.procPublicationRequest.oct-dec-2014.csv'
rows = pd.read_csv(fn, header=0, sep='|', encoding='latin-1')

In [ ]:
cols = ['RequestID', 'StartDate', 'EndDate',
        'AgencyCode', 'AgencyName', 'AgencyDivision',
        'TypeOfNoticeCode', 'TypeOfNoticeDescription',
        'ShortTitle', 'SectionID', 'SectionName',
        'DueDate', 'ConfirmationNumber',
        'AdditionalDescription']

In [ ]:
rows = rows[cols]

In [ ]:
# from IPython.display import display
# badcount = 0
# for html in rows.AdditionalDescription.values:
#     if not isinstance(html, str):
#         badcount += 1
#     else:
#         display(HTML(html))
#         try:
#             parse_notice(Soup(html))
#         except Exception:
#             badcount += 1

In [ ]:
import json
def scrape(row):
    output = None
    try:
        output = parse_notice(Soup(row.AdditionalDescription))
    except Exception :
        output = {'error' : 'bad input: [{}]'.format(row.AdditionalDescription)}
    row['output'] = json.dumps(output)
    return row

In [ ]:
mocs = rows['AgencyName'] == "Mayor's Office of Contract Services"
meets = rows['TypeOfNoticeDescription'] == "Notice"
fix = rows[mocs & meets].apply(scrape,1)


In [ ]:
for rec in fix[['output', 'AdditionalDescription']].values:
    output, text = rec
    if not isinstance(text, str):
        text = 'NAN'
    display(HTML(text))
    print('---')
    pprint.pprint(json.loads(output))
    display(HTML('<hr />'))
    print('\n\n')